In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random 

# Set the seed
random.seed(42)


# READ FILES

## OPEN MEDIC

Information specific to a client

In [2]:

# Read in the data
path_to_OpenMedic = "OpenMedic/OPEN_MEDIC_2022 2.CSV"
openMedic = pd.read_csv(path_to_OpenMedic, sep=";", encoding="ISO-8859-1")

print(openMedic.columns)
# openMedic.head()

Index(['ATC1', 'l_ATC1', 'ATC2', 'L_ATC2', 'ATC3', 'L_ATC3', 'ATC4', 'L_ATC4',
       'ATC5', 'L_ATC5', 'CIP13', 'l_cip13', 'TOP_GEN', 'GEN_NUM', 'age',
       'sexe', 'BEN_REG', 'PSP_SPE', 'BOITES', 'REM', 'BSE'],
      dtype='object')


In [3]:
openMedic = openMedic.rename(columns={"BEN_REG": "region_of_residence",
                                      "PSP_SPE": "prescriber",
                                      "BOITES": "number_of_boxes_delivered_per_client",
                                      "REM": "amount_reimbursed_per_client",
                                      "BSE": "reimbursement_base_per_client",})
openMedic.columns

Index(['ATC1', 'l_ATC1', 'ATC2', 'L_ATC2', 'ATC3', 'L_ATC3', 'ATC4', 'L_ATC4',
       'ATC5', 'L_ATC5', 'CIP13', 'l_cip13', 'TOP_GEN', 'GEN_NUM', 'age',
       'sexe', 'region_of_residence', 'prescriber',
       'number_of_boxes_delivered_per_client', 'amount_reimbursed_per_client',
       'reimbursement_base_per_client'],
      dtype='object')

In [4]:
openMedic.loc[openMedic['CIP13'] == 3400921609080]
openMedic['CIP13'].nunique()

12718

## MEDIC'AM

Information specific to a medicament

In [5]:
path_to_MedicAm = "Medic'AM/2022-medic-am-par-type-de-prescripteur_serie-annuelle.xls"
dict_df = pd.read_excel(path_to_MedicAm,sheet_name=['MedicAM_2022_tous_presc','MedicAM_2022_presc_ville','MedicAM_2022_presc_hopital'])

MedicAM_2022_presc_hopital = dict_df.get('MedicAM_2022_presc_hopital')
MedicAM_2022_presc_ville = dict_df.get('MedicAM_2022_presc_ville')
MedicAM_2022_tous_presc = dict_df.get('MedicAM_2022_tous_presc')

print(MedicAM_2022_tous_presc.columns)

Index(['CIP13', 'NOM COURT', 'PRODUIT', 'Code EphMRA', 'Classe EphMRA',
       'Code\nATC', 'Classe\nATC', 'Code\nATC 2', 'Libellé\nATC 2',
       'Base de remboursement 2022', 'Nombre de boites remboursées \n2022',
       'Montant remboursé \n2022'],
      dtype='object')


In [6]:
MedicAM_2022_tous_presc = MedicAM_2022_tous_presc.rename(columns={"Base de remboursement 2022": "reimbursement_base_2022",
                                                                    "Nombre de boites remboursées \n2022": "nb_boxes_reimbursed_2022",
                                                                    "Montant remboursé \n2022": "reimbursed_amount_2022",})
MedicAM_2022_tous_presc.columns

Index(['CIP13', 'NOM COURT', 'PRODUIT', 'Code EphMRA', 'Classe EphMRA',
       'Code\nATC', 'Classe\nATC', 'Code\nATC 2', 'Libellé\nATC 2',
       'reimbursement_base_2022', 'nb_boxes_reimbursed_2022',
       'reimbursed_amount_2022'],
      dtype='object')

In [7]:
MedicAM_2022_tous_presc.loc[MedicAM_2022_tous_presc['CIP13'] == 3400921609080]
MedicAM_2022_tous_presc['CIP13'].nunique()

15103

## CIS Files

In [8]:
path_to_DBB_Medicaments = "BDD_Medicaments"
list_path = []

for (dirpath, dirnames, filenames) in os.walk(path_to_DBB_Medicaments):
    list_path.extend(os.path.join(dirpath, filename) for filename in filenames)

list_df = {}
for path in list_path:
    df = pd.read_csv(path, encoding='latin-1', delimiter='\t',header=None)
    list_df[path.split('/')[-1]] = df
list_df.keys()

print('Columns of the dataframes : \n')
for key in list_df.keys():
    print(key ,':', list_df[key].columns)
    # print('-'*20)

Columns of the dataframes : 

CIS_COMPO_bdpm.txt : Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
CIS_bdpm.txt : Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')
CIS_CIP_bdpm.txt : Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64')
CIS_InfoImportantes_20240106103446_bdpm.txt : Index([0, 1, 2, 3], dtype='int64')
CIS_CIP_Dispo_Spec.txt : Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')
HAS_LiensPageCT_bdpm.txt : Index([0, 1], dtype='int64')
CIS_HAS_ASMR_bdpm.txt : Index([0, 1, 2, 3, 4, 5], dtype='int64')
CIS_HAS_SMR_bdpm.txt : Index([0, 1, 2, 3, 4, 5], dtype='int64')
CIS_GENER_bdpm.txt : Index([0, 1, 2, 3, 4, 5], dtype='int64')
CIS_CPD_bdpm.txt : Index([0, 1], dtype='int64')


We can see that the columns does not have names. We will replace the columns names by their real names to facilitate the utilisation.


Among these files we will be using `CIS_CIP_bdpm.txt`
Il contient la liste des présentations (boîtes de médicaments) disponibles pour les
médicaments présents dans le fichier décrit dans le paragraphe 3.1.
Les éléments mis à disposition dans ce fichier sont les suivants :
- Code CIS
- Code CIP7 (Code Identifiant de Présentation à 7 chiffres)
- Libellé de la présentation
- Statut administratif de la présentation
- Etat de commercialisation de la présentation tel que déclaré par le titulaire de l'AMM
- Date de la déclaration de commercialisation (format JJ/MM/AAAA)
- Code CIP13 (Code Identifiant de Présentation à 13 chiffres)
- Agrément aux collectivités ("oui", "non" ou « inconnu »)
- Taux de remboursement (avec un séparateur « ; » entre chaque valeur quand il y en a
plusieurs)
- Prix du médicament en euro
- Texte présentant les indications ouvrant droit au remboursement par l’assurance
maladie s’il y a plusieurs taux de remboursement pour la même présentation.

In [9]:
list_df['CIS_CIP_bdpm.txt'] = list_df['CIS_CIP_bdpm.txt'].rename(columns={0:"CIS",
                                                                          1:"CIP7",
                                                                          2:"description_medication",
                                                                          3:"statut",
                                                                          4:"commercialisation_state_by_owner",
                                                                          5:"date_declaration_commercialisation",
                                                                          6:"CIP13",
                                                                          7:"collectivity_agreement",
                                                                          8:"reimbursement_rate_per_medication",
                                                                          9:"price_medication",
                                                                          10:10,
                                                                          11:11,
                                                                          12:"pharmacy_recommendation",})


In [10]:
list_df['CIS_bdpm.txt'] = list_df['CIS_bdpm.txt'].rename(columns={0:"CIS",
                                                                    1:"name_medication",
                                                                    2:"pharmaceutical_form",
                                                                    3:"methods_of_administration",
                                                                    4:"statut_amm",
                                                                    5:"procedure_type_amm",
                                                                    6:"commercialisation_state",
                                                                    7:"date_AMM",
                                                                    8:"statutBdm",
                                                                    9:"european_authorization_number",
                                                                    10:"owner",
                                                                    11:"enhanced_surveillance"})
list_df['CIS_bdpm.txt'].drop(columns=['statutBdm'], inplace=True)

We merge our files on the CIS code to gather all useful information to one dataframe

In [11]:
master_cis_files = list_df['CIS_CIP_bdpm.txt'].merge(list_df['CIS_bdpm.txt'], on='CIS', how='left')

In [12]:
master_cis_files.loc[master_cis_files['CIP13'] == 3400921609080]
master_cis_files['CIP13'].nunique()

20857

In [13]:
master_dataframe = openMedic.merge(
    master_cis_files, on='CIP13', how='left').merge(
    MedicAM_2022_tous_presc[['CIP13','reimbursement_base_2022', 'nb_boxes_reimbursed_2022','reimbursed_amount_2022']], on='CIP13', how='left')

In [14]:
master_dataframe['CIP13'].nunique()

12718

In [15]:
master_dataframe.columns

Index([                                'ATC1',
                                     'l_ATC1',
                                       'ATC2',
                                     'L_ATC2',
                                       'ATC3',
                                     'L_ATC3',
                                       'ATC4',
                                     'L_ATC4',
                                       'ATC5',
                                     'L_ATC5',
                                      'CIP13',
                                    'l_cip13',
                                    'TOP_GEN',
                                    'GEN_NUM',
                                        'age',
                                       'sexe',
                        'region_of_residence',
                                 'prescriber',
       'number_of_boxes_delivered_per_client',
               'amount_reimbursed_per_client',
              'reimbursement_base_per_client',
             

In [16]:
master_dataframe['master_key'] = master_dataframe.index
master_dataframe.head()

,ATC1,l_ATC1,ATC2,L_ATC2,ATC3,L_ATC3,ATC4,L_ATC4,ATC5,L_ATC5,...,procedure_type_amm,commercialisation_state,date_AMM,european_authorization_number,owner,enhanced_surveillance,reimbursement_base_2022,nb_boxes_reimbursed_2022,reimbursed_amount_2022,master_key
0,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,Procï¿½dure nationale,Non commercialisï¿½e,10/09/1992,NaN,ROTTAPHARM,Non,420.7,316.0,140.0775,0
1,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,Procï¿½dure nationale,Non commercialisï¿½e,10/09/1992,NaN,ROTTAPHARM,Non,420.7,316.0,140.0775,1
2,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,Procï¿½dure nationale,Non commercialisï¿½e,10/09/1992,NaN,ROTTAPHARM,Non,420.7,316.0,140.0775,2
3,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,Procï¿½dure nationale,Non commercialisï¿½e,10/09/1992,NaN,ROTTAPHARM,Non,420.7,316.0,140.0775,3
4,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,Procï¿½dure nationale,Non commercialisï¿½e,10/09/1992,NaN,ROTTAPHARM,Non,420.7,316.0,140.0775,4


# Table creation 

![diagram](logical_model.drawio.png)

##  `beneficiary`

In [17]:
import random

def age_group_to_age(age_group):
    if age_group == 0:
        return random.randint(1,19)
    elif age_group == 20:
        return random.randint(20,59)
    elif age_group == 60:
        return random.randint(60,100)
    else:
        return random.randint(1,100)

def nbSexe_to_strSexe(sexe):
    if sexe == 1:
        return "MASCULIN"
    elif sexe == 2:
        return "FEMININ"
    else:
        # return masculin or feminin randomly
        return random.choice(["MASCULIN","FEMININ"])
    
def nbRegion_to_strRegion(region):
    if region == 5:
        return "Régions et Départements d'outre-mer"
    elif region == 11:
        return "Ile-de-France"
    elif region == 24:
        return "Centre-Val de Loire"
    elif region == 27:
        return "Bourgogne-Franche-Comté"
    elif region == 28:
        return "Normandie"
    elif region == 32:
        return "Nord-Pas-de-Calais-Picardie"
    elif region == 44:
        return "Alsace-Champagne-Ardenne-Lorraine"
    elif region == 52:
        return "Pays de la Loire"
    elif region == 53:
        return "Bretagne"
    elif region == 75:
        return "Aquitaine-Limousin-Poitou-Charentes"
    elif region == 76:
        return "Languedoc-Roussillon-Midi-Pyrénées"
    elif region == 84:
        return "Auvergne-Rhône-Alpes"
    elif region == 93:
        return "Provence-Alpes-Côte d'Azur et Corse"
    elif region == 0 or region == 99:
        return "Inconnu"
    else:
        return random.choice(["Régions et Départements d'outre-mer","Ile-de-France","Centre-Val de Loire","Bourgogne-Franche-Comté","Normandie","Nord-Pas-de-Calais-Picardie","Alsace-Champagne-Ardenne-Lorraine","Pays de la Loire","Bretagne","Aquitaine-Limousin-Poitou-Charentes","Languedoc-Roussillon-Midi-Pyrénées","Auvergne-Rhône-Alpes","Provence-Alpes-Côte d'Azur et Corse","Inconnu"])

In [18]:
beneficiary = master_dataframe[['region_of_residence','age','sexe','master_key']].copy()
beneficiary['age'] = beneficiary['age'].apply(age_group_to_age)
beneficiary['beneficiary_id'] = beneficiary.index
beneficiary.shape,beneficiary.head()

((1873493, 5),
    region_of_residence  age  sexe  master_key  beneficiary_id
 0                    5    4     1           0               0
 1                   11    1     1           1               1
 2                   44    9     1           2               2
 3                   84    8     1           3               3
 4                   99    8     1           4               4)

##  `Medication_specialisation`

In [19]:
# I need to create a table with the following columns :
# - 'medicament_specialisation_id' : primary key
# - beneficiary_id : foreign key
# - CIP13 : foreign key
# - ATC1
# - ATC2
# - ATC3
# - ATC4
# - ATC5
# - l_ATC1
# - L_ATC2
# - L_ATC3
# - L_ATC4
# - L_ATC5
# - prescriber
# - number_of_boxes_delivered_per_client


medication_specialisation = pd.merge(beneficiary[['beneficiary_id', 'master_key']], 
                      master_dataframe[['CIP13', 'ATC1','ATC2','ATC3','ATC4','ATC5','l_ATC1','L_ATC2','L_ATC3','L_ATC4','L_ATC5','prescriber','number_of_boxes_delivered_per_client','master_key']], 
                      on='master_key', 
                      how='inner')

medication_specialisation['medicament_specialisation_id'] = medication_specialisation.index


medication_specialisation.shape,medication_specialisation.head()


((1873493, 16),
    beneficiary_id  master_key          CIP13 ATC1 ATC2  ATC3   ATC4     ATC5  \
 0               0           0  3400931911999    A  A01  A01A  A01AA  A01AA01   
 1               1           1  3400931911999    A  A01  A01A  A01AA  A01AA01   
 2               2           2  3400931911999    A  A01  A01A  A01AA  A01AA01   
 3               3           3  3400931911999    A  A01  A01A  A01AA  A01AA01   
 4               4           4  3400931911999    A  A01  A01A  A01AA  A01AA01   
 
                             l_ATC1                        L_ATC2  \
 0  SYSTEME DIGESTIF ET METABOLISME  PREPARATIONS STOMATOLOGIQUES   
 1  SYSTEME DIGESTIF ET METABOLISME  PREPARATIONS STOMATOLOGIQUES   
 2  SYSTEME DIGESTIF ET METABOLISME  PREPARATIONS STOMATOLOGIQUES   
 3  SYSTEME DIGESTIF ET METABOLISME  PREPARATIONS STOMATOLOGIQUES   
 4  SYSTEME DIGESTIF ET METABOLISME  PREPARATIONS STOMATOLOGIQUES   
 
                          L_ATC3                                  L_ATC4  \
 0  

# `Medication_presentation`

In [20]:
# I need to create a table with the following columns :
### - CIP13 : primary key
### - name_medication
# - pharmaceutical_form
# - methods_of_administration
# - statut_amm
# - procedure_type_amm
### - commercialisation_state
# - date_AMM
### - statutBdm
# - european_authorization_number
### - owner
### - enhanced_surveillance
# - description_medication
# - commercialisation_state_by_owner
### - date_declaration_commercialisation
### - reimbursement_rate_per_medication
### - price_medication
# - reimbursement_base_2022
# - nb_boxes_reimbursed_2022
# - reimbursed_amount_2022

master_dataframe_unique = master_dataframe.drop_duplicates(subset=['CIP13'])


medication_presentation = pd.merge(master_cis_files[['CIP13','name_medication','pharmaceutical_form','methods_of_administration','statut_amm','procedure_type_amm','commercialisation_state','date_AMM','owner','enhanced_surveillance','description_medication','date_declaration_commercialisation','reimbursement_rate_per_medication','price_medication']], 
                      master_dataframe_unique['CIP13'], 
                      on='CIP13', 
                      how='inner')

# medication_presentation = master_cis_files[['CIP13','name_medication','pharmaceutical_form','methods_of_administration','statut_amm','procedure_type_amm','commercialisation_state','date_AMM ','statutBdm ','owner','enhanced_surveillance','description_medication','date_declaration_commercialisation','reimbursement_rate_per_medication','price_medication']].copy()
medication_presentation.shape,medication_presentation.head()


((12174, 14),
            CIP13                                    name_medication  \
 0  3400949497294    ANASTROZOLE ACCORD 1 mg, comprimï¿½ pelliculï¿½   
 1  3400949497706    ANASTROZOLE ACCORD 1 mg, comprimï¿½ pelliculï¿½   
 2  3400936963504  BECLOSPIN 800 microgrammes/2ml suspension pour...   
 3  3400936145825                  FENOFIBRATE TEVA 100 mg, gï¿½lule   
 4  3400939725192  TRAMADOL EG L.P. 200 mg, comprimï¿½ ï¿½ libï¿½...   
 
                            pharmaceutical_form methods_of_administration  \
 0                       comprimï¿½ pelliculï¿½                     orale   
 1                       comprimï¿½ pelliculï¿½                     orale   
 2  suspension pour inhalation par nï¿½buliseur                 inhalï¿½e   
 3                                     gï¿½lule                     orale   
 4      comprimï¿½ ï¿½ libï¿½ration prolongï¿½e                     orale   
 
             statut_amm                      procedure_type_amm  \
 0  Autorisation acti

## `Global_expenses`

In [21]:
# I need to create a table with the following columns :
### - global_expenses_id: primary key
### - CIP13 : foreign key
# - name_medication
# - reimbursement_base_2022
# - nb_boxes_reimbursed_2022
# - reimbursed_amount_2022


global_expenses = pd.merge(MedicAM_2022_tous_presc[['CIP13','reimbursement_base_2022','nb_boxes_reimbursed_2022','reimbursed_amount_2022']], 
                      master_cis_files[['CIP13','name_medication']], 
                      on='CIP13', 
                      how='inner').merge(master_dataframe_unique['CIP13'], 
                                        on='CIP13', 
                                        how='inner')


global_expenses['global_expenses_id'] = global_expenses.index

global_expenses.shape,global_expenses.head()


((12174, 6),
            CIP13  reimbursement_base_2022  nb_boxes_reimbursed_2022  \
 0  3400921604696                353956.43                     96127   
 1  3400921609080              16032399.34                     25220   
 2  3400921610550                 55389.28                     20437   
 3  3400921610840                160428.96                     26086   
 4  3400921611731                306520.92                     89626   
 
    reimbursed_amount_2022                                    name_medication  \
 0            2.450826e+05              YELLOX 0,9 mg/ml, collyre en solution   
 1            1.588837e+07  AVONEX 30 microgrammes/0,5 ml, solution inject...   
 2            9.807795e+03                  POLYGYNAX VIRGO, capsule vaginale   
 3            1.573739e+05                NICORANDIL SANDOZ 20 mg, comprimï¿½   
 4            2.974557e+05      NICORANDIL SANDOZ 10 mg, comprimï¿½ sï¿½cable   
 
    global_expenses_id  
 0                   0  
 1             

## `Generic_group`

In [22]:
# I need to create a table with the following columns :
### - generic_group_id: primary key
### - CIP13 : foreign key
# - GEN_NUM
# - TOP_GEN

openMedic_unique = openMedic.drop_duplicates(subset=['CIP13'])


generic_group = pd.merge(openMedic_unique[['CIP13','GEN_NUM','TOP_GEN']],
                      master_dataframe_unique['CIP13'], 
                      on='CIP13', 
                      how='inner').merge(master_cis_files[['CIP13']],on='CIP13',how='inner')



generic_group['generic_group_id'] = generic_group.index
generic_group.shape,generic_group.head()

((12174, 4),
            CIP13  GEN_NUM  TOP_GEN  generic_group_id
 0  3400931911999        0        0                 0
 1  3400932409860        0        0                 1
 2  3400932742417        0        0                 2
 3  3400934089480        0        0                 3
 4  3400934780509        0        0                 4)

In [23]:
tables = {
    'beneficiary': beneficiary,
    'medication_specialisation': medication_specialisation,
    'medication_presentation': medication_presentation,
    'global_expenses': global_expenses,
    'generic_group': generic_group
}

In [24]:
for name, table in tables.items():
    if 'master_key' in table.columns: 
        table.drop(columns=['master_key'],inplace=True)

In [25]:
for name, table in tables.items():
    print(name,table.shape)


beneficiary (1873493, 4)
medication_specialisation (1873493, 15)
medication_presentation (12174, 14)
global_expenses (12174, 6)
generic_group (12174, 4)


In [26]:
for name, table in tables.items():
    print(name, '\n',table.columns,'\n',table.dtypes)
    print('--------'*10,'\n')

beneficiary 
 Index(['region_of_residence', 'age', 'sexe', 'beneficiary_id'], dtype='object') 
 region_of_residence    int64
age                    int64
sexe                   int64
beneficiary_id         int64
dtype: object
-------------------------------------------------------------------------------- 

medication_specialisation 
 Index(['beneficiary_id', 'CIP13', 'ATC1', 'ATC2', 'ATC3', 'ATC4', 'ATC5',
       'l_ATC1', 'L_ATC2', 'L_ATC3', 'L_ATC4', 'L_ATC5', 'prescriber',
       'number_of_boxes_delivered_per_client', 'medicament_specialisation_id'],
      dtype='object') 
 beneficiary_id                           int64
CIP13                                   object
ATC1                                    object
ATC2                                    object
ATC3                                    object
ATC4                                    object
ATC5                                    object
l_ATC1                                  object
L_ATC2                                

We now have all our usefull data. We will then preprocess them before storing into the database.

## Preprocessing

1. Handle Missing Values

2. Data Type Conversion



Finally, store the preprocessed DataFrame into the database


First we will handle missing values.

In [27]:
for name, table in tables.items():
    print(table.isna().sum()) 
    table.fillna(0, inplace=True)  # Fill missing numerical values with 0

region_of_residence    0
age                    0
sexe                   0
beneficiary_id         0
dtype: int64
beneficiary_id                             0
CIP13                                      0
ATC1                                       0
ATC2                                       0
ATC3                                       0
ATC4                                       0
ATC5                                       0
l_ATC1                                     0
L_ATC2                                    18
L_ATC3                                     0
L_ATC4                                  5165
L_ATC5                                     0
prescriber                                 0
number_of_boxes_delivered_per_client       0
medicament_specialisation_id               0
dtype: int64
CIP13                                   0
name_medication                         0
pharmaceutical_form                     0
methods_of_administration               0
statut_amm                     

In [28]:
for name, table in tables.items():    # Select columns of type 'object'
    object_columns = table.select_dtypes(include=['object'])
    
    # Convert values in object columns to lowercase
    table[object_columns.columns] = object_columns.map(lambda x: x.lower() if isinstance(x, str) else x)

    # Update the DataFrame in the tables dictionary
    tables[name] = table

Here we lowercase all str instances in our tables.

#### `beneficiary`

In [29]:
tables['beneficiary'].head(n=5), print(tables['beneficiary'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873493 entries, 0 to 1873492
Data columns (total 4 columns):
 #   Column               Dtype
---  ------               -----
 0   region_of_residence  int64
 1   age                  int64
 2   sexe                 int64
 3   beneficiary_id       int64
dtypes: int64(4)
memory usage: 57.2 MB
None


(   region_of_residence  age  sexe  beneficiary_id
 0                    5    4     1               0
 1                   11    1     1               1
 2                   44    9     1               2
 3                   84    8     1               3
 4                   99    8     1               4,
 None)

- For the `beneficiary` table we have only values of type int so we do not have to make any preprocessing

#### `medication_specialisation`

In [30]:
print(tables['medication_specialisation'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873493 entries, 0 to 1873492
Data columns (total 15 columns):
 #   Column                                Dtype 
---  ------                                ----- 
 0   beneficiary_id                        int64 
 1   CIP13                                 int64 
 2   ATC1                                  object
 3   ATC2                                  object
 4   ATC3                                  object
 5   ATC4                                  object
 6   ATC5                                  object
 7   l_ATC1                                object
 8   L_ATC2                                object
 9   L_ATC3                                object
 10  L_ATC4                                object
 11  L_ATC5                                object
 12  prescriber                            int64 
 13  number_of_boxes_delivered_per_client  int64 
 14  medicament_specialisation_id          int64 
dtypes: int64(5), object(10)
memory u

We can see that we have int and object type for this table. Object type representing string.

In [31]:
# Get columns with data type 'object'
object_columns = tables['medication_specialisation'].select_dtypes(include=['object'])

# Print information about these columns
print(object_columns.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873493 entries, 0 to 1873492
Data columns (total 10 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   ATC1    object
 1   ATC2    object
 2   ATC3    object
 3   ATC4    object
 4   ATC5    object
 5   l_ATC1  object
 6   L_ATC2  object
 7   L_ATC3  object
 8   L_ATC4  object
 9   L_ATC5  object
dtypes: object(10)
memory usage: 142.9+ MB
None


Now let dive a bit into this features to see how we can preprocess them.

In [32]:
tables['medication_specialisation'].head(n=5)

,beneficiary_id,CIP13,ATC1,ATC2,ATC3,ATC4,ATC5,l_ATC1,L_ATC2,L_ATC3,L_ATC4,L_ATC5,prescriber,number_of_boxes_delivered_per_client,medicament_specialisation_id
0,0,3400931911999,a,a01,a01a,a01aa,a01aa01,systeme digestif et metabolisme,preparations stomatologiques,preparations stomatologiques,medicaments prophylactiques anticaries,sodium fluorure,99,17,0
1,1,3400931911999,a,a01,a01a,a01aa,a01aa01,systeme digestif et metabolisme,preparations stomatologiques,preparations stomatologiques,medicaments prophylactiques anticaries,sodium fluorure,99,24,1
2,2,3400931911999,a,a01,a01a,a01aa,a01aa01,systeme digestif et metabolisme,preparations stomatologiques,preparations stomatologiques,medicaments prophylactiques anticaries,sodium fluorure,99,20,2
3,3,3400931911999,a,a01,a01a,a01aa,a01aa01,systeme digestif et metabolisme,preparations stomatologiques,preparations stomatologiques,medicaments prophylactiques anticaries,sodium fluorure,99,13,3
4,4,3400931911999,a,a01,a01a,a01aa,a01aa01,systeme digestif et metabolisme,preparations stomatologiques,preparations stomatologiques,medicaments prophylactiques anticaries,sodium fluorure,1,14,4


#### `medication_presentation`


In [33]:
print(tables['medication_presentation'].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12174 entries, 0 to 12173
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   CIP13                               12174 non-null  int64 
 1   name_medication                     12174 non-null  object
 2   pharmaceutical_form                 12174 non-null  object
 3   methods_of_administration           12174 non-null  object
 4   statut_amm                          12174 non-null  object
 5   procedure_type_amm                  12174 non-null  object
 6   commercialisation_state             12174 non-null  object
 7   date_AMM                            12174 non-null  object
 8   owner                               12174 non-null  object
 9   enhanced_surveillance               12174 non-null  object
 10  description_medication              12174 non-null  object
 11  date_declaration_commercialisation  12174 non-null  ob

In [34]:
# Get columns with data type 'object'
object_columns = tables['medication_presentation'].select_dtypes(include=['object'])

# Print information about these columns
print(object_columns.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12174 entries, 0 to 12173
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   name_medication                     12174 non-null  object
 1   pharmaceutical_form                 12174 non-null  object
 2   methods_of_administration           12174 non-null  object
 3   statut_amm                          12174 non-null  object
 4   procedure_type_amm                  12174 non-null  object
 5   commercialisation_state             12174 non-null  object
 6   date_AMM                            12174 non-null  object
 7   owner                               12174 non-null  object
 8   enhanced_surveillance               12174 non-null  object
 9   description_medication              12174 non-null  object
 10  date_declaration_commercialisation  12174 non-null  object
 11  reimbursement_rate_per_medication   12174 non-null  ob

In [35]:
tables['medication_presentation'].head(n=5)

,CIP13,name_medication,pharmaceutical_form,methods_of_administration,statut_amm,procedure_type_amm,commercialisation_state,date_AMM,owner,enhanced_surveillance,description_medication,date_declaration_commercialisation,reimbursement_rate_per_medication,price_medication
0,3400949497294,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,28/10/2010,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 30 comprimé(s),16/03/2011,100%,"30,36"
1,3400949497706,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,28/10/2010,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 90 comprimé(s),19/09/2011,100%,"85,74"
2,3400936963504,beclospin 800 microgrammes/2ml suspension pour...,suspension pour inhalation par nï¿½buliseur,inhalï¿½e,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,06/01/2006,chiesi,non,20 récipient(s) unidose(s) polyéthylène de 2 m...,30/11/2006,65%,"16,77"
3,3400936145825,"fenofibrate teva 100 mg, gï¿½lule",gï¿½lule,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,06/12/1996,teva sante,non,plaquette(s) thermoformée(s) pvc aluminium de ...,11/06/2003,65%,"1,54"
4,3400939725192,"tramadol eg l.p. 200 mg, comprimï¿½ ï¿½ libï¿½...",comprimï¿½ ï¿½ libï¿½ration prolongï¿½e,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,07/09/2009,eg labo - laboratoires eurogenerics,non,plaquette(s) opaque(s) pvc-aluminium de 30 com...,12/12/2011,65%,"8,88"


We have some caracteres that we could not decode properly so we will handle this. Also we can see that reimbursement_rate_per_medication and price_medication are type object instead of being float. 

In [36]:
tables['medication_presentation'][['CIP13']] = tables['medication_presentation'][['CIP13']].astype(int)  # Convert beneficiary_id to int

In [37]:
def string_pourcenatge_to_float(input_str):
    # Replace ',' with '.'
    if isinstance(input_str, str):
        formatted_str = input_str.replace(',', '.').strip('%')

        # Convert to float
        result = float(formatted_str)

        return result

In [38]:
tables['medication_presentation']['reimbursement_rate_per_medication'] =tables['medication_presentation']['reimbursement_rate_per_medication'].apply(string_pourcenatge_to_float)

In [39]:
def string_to_float(input_str):
    # Replace ',' with '.'

    if isinstance(input_str, str):
        formatted_str = input_str.replace(',', '.')

        # Check if the string contains a period
        if '.' in formatted_str:
            # Remove all '.' characters except the last one
            formatted_str = formatted_str.replace('.', '', formatted_str.count('.') - 1)

        # Convert to float
        result = float(formatted_str)

        return result


In [40]:

tables['medication_presentation']['price_medication'] = tables['medication_presentation']['price_medication'].apply(string_to_float)

In [41]:
tables['medication_presentation'][['price_medication','reimbursement_rate_per_medication']] = tables['medication_presentation'][['price_medication','reimbursement_rate_per_medication']].astype(float)  # Convert beneficiary_id to int

In [42]:
print(tables['medication_presentation'].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12174 entries, 0 to 12173
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CIP13                               12174 non-null  int64  
 1   name_medication                     12174 non-null  object 
 2   pharmaceutical_form                 12174 non-null  object 
 3   methods_of_administration           12174 non-null  object 
 4   statut_amm                          12174 non-null  object 
 5   procedure_type_amm                  12174 non-null  object 
 6   commercialisation_state             12174 non-null  object 
 7   date_AMM                            12174 non-null  object 
 8   owner                               12174 non-null  object 
 9   enhanced_surveillance               12174 non-null  object 
 10  description_medication              12174 non-null  object 
 11  date_declaration_commercialisation  12174

In [43]:
tables['medication_presentation'].head(n=5)

,CIP13,name_medication,pharmaceutical_form,methods_of_administration,statut_amm,procedure_type_amm,commercialisation_state,date_AMM,owner,enhanced_surveillance,description_medication,date_declaration_commercialisation,reimbursement_rate_per_medication,price_medication
0,3400949497294,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,28/10/2010,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 30 comprimé(s),16/03/2011,100.0,30.36
1,3400949497706,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,28/10/2010,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 90 comprimé(s),19/09/2011,100.0,85.74
2,3400936963504,beclospin 800 microgrammes/2ml suspension pour...,suspension pour inhalation par nï¿½buliseur,inhalï¿½e,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,06/01/2006,chiesi,non,20 récipient(s) unidose(s) polyéthylène de 2 m...,30/11/2006,65.0,16.77
3,3400936145825,"fenofibrate teva 100 mg, gï¿½lule",gï¿½lule,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,06/12/1996,teva sante,non,plaquette(s) thermoformée(s) pvc aluminium de ...,11/06/2003,65.0,1.54
4,3400939725192,"tramadol eg l.p. 200 mg, comprimï¿½ ï¿½ libï¿½...",comprimï¿½ ï¿½ libï¿½ration prolongï¿½e,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,07/09/2009,eg labo - laboratoires eurogenerics,non,plaquette(s) opaque(s) pvc-aluminium de 30 com...,12/12/2011,65.0,8.88


In [44]:
# Example of date columns in the DataFrame
date_columns = ['date_AMM', 'date_declaration_commercialisation']

# Convert date columns to datetime data type
for column in date_columns:
    tables['medication_presentation'][column] = pd.to_datetime(tables['medication_presentation'][column])

/var/folders/3j/0jwg1_4n3g7dtvqn4hk8cwy80000gn/T/ipykernel_36497/1131401778.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tables['medication_presentation'][column] = pd.to_datetime(tables['medication_presentation'][column])
/var/folders/3j/0jwg1_4n3g7dtvqn4hk8cwy80000gn/T/ipykernel_36497/1131401778.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tables['medication_presentation'][column] = pd.to_datetime(tables['medication_presentation'][column])


In [45]:
tables['medication_presentation'].head(n=5)

,CIP13,name_medication,pharmaceutical_form,methods_of_administration,statut_amm,procedure_type_amm,commercialisation_state,date_AMM,owner,enhanced_surveillance,description_medication,date_declaration_commercialisation,reimbursement_rate_per_medication,price_medication
0,3400949497294,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,2010-10-28,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 30 comprimé(s),2011-03-16,100.0,30.36
1,3400949497706,"anastrozole accord 1 mg, comprimï¿½ pelliculï¿½",comprimï¿½ pelliculï¿½,orale,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,2010-10-28,accord healthcare france,non,plaquette(s) pvc pvdc aluminium de 90 comprimé(s),2011-09-19,100.0,85.74
2,3400936963504,beclospin 800 microgrammes/2ml suspension pour...,suspension pour inhalation par nï¿½buliseur,inhalï¿½e,autorisation active,procï¿½dure de reconnaissance mutuelle,commercialisï¿½e,2006-01-06,chiesi,non,20 récipient(s) unidose(s) polyéthylène de 2 m...,2006-11-30,65.0,16.77
3,3400936145825,"fenofibrate teva 100 mg, gï¿½lule",gï¿½lule,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,1996-12-06,teva sante,non,plaquette(s) thermoformée(s) pvc aluminium de ...,2003-06-11,65.0,1.54
4,3400939725192,"tramadol eg l.p. 200 mg, comprimï¿½ ï¿½ libï¿½...",comprimï¿½ ï¿½ libï¿½ration prolongï¿½e,orale,autorisation active,procï¿½dure nationale,commercialisï¿½e,2009-09-07,eg labo - laboratoires eurogenerics,non,plaquette(s) opaque(s) pvc-aluminium de 30 com...,2011-12-12,65.0,8.88


In [46]:
import re

def normalize_string(string_with_special_chars):
    """
    Normalize a string by handling special characters and removing non-alphanumeric characters.

    Parameters:
    - string_with_special_chars (str): Input string containing special characters.

    Returns:
    - str: Normalized string with special characters removed.
    """
    if isinstance(string_with_special_chars,str):
        # Normalize the string by ignoring errors during encoding
        normalized_string = string_with_special_chars.encode('utf-8', errors='ignore').decode('utf-8')

        # Split the string into words
        words = normalized_string.split()

        # Remove non-alphanumeric characters from words
        cleaned_words = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in words]

        # Join the cleaned words back into a single string
        cleaned_string = ' '.join(cleaned_words)
    else: 
        return '0'

    return cleaned_string


In [47]:
# Apply the normalize_string function to each column containing strings with special characters

object_columns = tables['medication_presentation'].select_dtypes(include=['object'])

for column in object_columns:
    print(f"Normalizing values in the {column} column")
    tables['medication_presentation'][column] = tables['medication_presentation'][column].apply(normalize_string)


Normalizing values in the name_medication column
Normalizing values in the pharmaceutical_form column
Normalizing values in the methods_of_administration column
Normalizing values in the statut_amm column
Normalizing values in the procedure_type_amm column
Normalizing values in the commercialisation_state column
Normalizing values in the owner column
Normalizing values in the enhanced_surveillance column
Normalizing values in the description_medication column


In [48]:
tables['medication_presentation'].head(n=5)

,CIP13,name_medication,pharmaceutical_form,methods_of_administration,statut_amm,procedure_type_amm,commercialisation_state,date_AMM,owner,enhanced_surveillance,description_medication,date_declaration_commercialisation,reimbursement_rate_per_medication,price_medication
0,3400949497294,anastrozole accord 1 mg comprim pellicul,comprim pellicul,orale,autorisation active,procdure de reconnaissance mutuelle,commercialise,2010-10-28,accord healthcare france,non,plaquettes pvc pvdc aluminium de 30 comprims,2011-03-16,100.0,30.36
1,3400949497706,anastrozole accord 1 mg comprim pellicul,comprim pellicul,orale,autorisation active,procdure de reconnaissance mutuelle,commercialise,2010-10-28,accord healthcare france,non,plaquettes pvc pvdc aluminium de 90 comprims,2011-09-19,100.0,85.74
2,3400936963504,beclospin 800 microgrammes2ml suspension pour ...,suspension pour inhalation par nbuliseur,inhale,autorisation active,procdure de reconnaissance mutuelle,commercialise,2006-01-06,chiesi,non,20 rcipients unidoses polythylne de 2 ml surem...,2006-11-30,65.0,16.77
3,3400936145825,fenofibrate teva 100 mg glule,glule,orale,autorisation active,procdure nationale,commercialise,1996-12-06,teva sante,non,plaquettes thermoformes pvc aluminium de 30 gl...,2003-06-11,65.0,1.54
4,3400939725192,tramadol eg lp 200 mg comprim libration prolonge,comprim libration prolonge,orale,autorisation active,procdure nationale,commercialise,2009-09-07,eg labo laboratoires eurogenerics,non,plaquettes opaques pvcaluminium de 30 comprims,2011-12-12,65.0,8.88


We have set our date in datetime format, our string are lowercased and encoded as utf-8 and our int and float are set also.

#### `global_expenses`


In [49]:
print(tables['global_expenses'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12174 entries, 0 to 12173
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CIP13                     12174 non-null  int64  
 1   reimbursement_base_2022   12174 non-null  float64
 2   nb_boxes_reimbursed_2022  12174 non-null  int64  
 3   reimbursed_amount_2022    12174 non-null  float64
 4   name_medication           12174 non-null  object 
 5   global_expenses_id        12174 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 570.8+ KB
None


In [50]:
tables['global_expenses'].head(n=5)

,CIP13,reimbursement_base_2022,nb_boxes_reimbursed_2022,reimbursed_amount_2022,name_medication,global_expenses_id
0,3400921604696,353956.43,96127,2.450826e+05,"yellox 0,9 mg/ml, collyre en solution",0
1,3400921609080,16032399.34,25220,1.588837e+07,"avonex 30 microgrammes/0,5 ml, solution inject...",1
2,3400921610550,55389.28,20437,9.807795e+03,"polygynax virgo, capsule vaginale",2
3,3400921610840,160428.96,26086,1.573739e+05,"nicorandil sandoz 20 mg, comprimï¿½",3
4,3400921611731,306520.92,89626,2.974557e+05,"nicorandil sandoz 10 mg, comprimï¿½ sï¿½cable",4


In [51]:
# Apply the normalize_string function to each column containing strings with special characters

object_columns = tables['global_expenses'].select_dtypes(include=['object'])

for column in object_columns:
    print(f"Normalizing values in the {column} column")
    tables['global_expenses'][column] = tables['global_expenses'][column].apply(normalize_string)

Normalizing values in the name_medication column


We normalize our str instances.

#### `generic_group`

In [52]:
print(tables['generic_group'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12174 entries, 0 to 12173
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   CIP13             12174 non-null  int64
 1   GEN_NUM           12174 non-null  int64
 2   TOP_GEN           12174 non-null  int64
 3   generic_group_id  12174 non-null  int64
dtypes: int64(4)
memory usage: 380.6 KB
None


Nothing to change for this table.

# Add our tables into our database

We created a postgres database hosted on AWS server. We will first connect to it and then 

In [68]:
import psycopg2
import sys
import boto3
import os

ENDPOINT="database-1.c3gcywo2moej.eu-north-1.rds.amazonaws.com"
PORT="5432"
USER="postgres"
REGION="eu-north-1c"
DBNAME="Data_test"
PASSWORD = 'motdepasse'


conn_string = "host="+ ENDPOINT +" port="+ PORT +" user=" + USER +" password=" + PASSWORD

try:
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    # cur.execute("""SELECT now()""")
    # query_results = cur.fetchall()
    print("Connetected to RDS")
except Exception as e:
    print("Database connection failed due to {}".format(e))   

Connetected to RDS


In [69]:
from sqlalchemy import create_engine

# Assuming your DataFrame is named 'df' and the table is 'client'


# Replace 'your_database_url' with the actual connection string for your PostgreSQL database
database_url = 'postgresql://'+ USER +':' + PASSWORD + '@' + ENDPOINT +':' + PORT + '/' + DBNAME
engine = create_engine(database_url)


tables = {
    'beneficiary': beneficiary,
    'medication_specialisation': medication_specialisation,
    'medication_presentation': medication_presentation,
    'global_expenses': global_expenses,
    'generic_group': generic_group
}

In [71]:
import pandas as pd

for name,table in tables.items():
        
    # Assuming you have a DataFrame called 'df' with your data
    # Calculate memory usage of the DataFrame in bytes
    memory_usage_bytes = table.memory_usage(deep=True).sum()

    # Convert bytes to megabytes (optional)
    memory_usage_megabytes = memory_usage_bytes / (1024**2)

    print("Memory usage of DataFrame {}: {:.2f} MB".format(name,memory_usage_megabytes))


Memory usage of DataFrame beneficiary: 57.17 MB
Memory usage of DataFrame medication_specialisation: 1378.82 MB
Memory usage of DataFrame medication_presentation: 8.72 MB
Memory usage of DataFrame global_expenses: 1.66 MB
Memory usage of DataFrame generic_group: 0.37 MB


In [77]:
import pandas as pd
from sqlalchemy import create_engine

for name,table in tables.items():
    # Convert the DataFrame to SQL table with specified data types
    %time table.to_sql(name, engine, if_exists='replace', index=False)


CPU times: user 27.3 s, sys: 1.24 s, total: 28.5 s
Wall time: 2min 35s
CPU times: user 1min 6s, sys: 12.8 s, total: 1min 19s
Wall time: 5min 43s
CPU times: user 648 ms, sys: 44.4 ms, total: 693 ms
Wall time: 2.85 s
CPU times: user 310 ms, sys: 14.1 ms, total: 324 ms
Wall time: 1.79 s
CPU times: user 276 ms, sys: 35.3 ms, total: 311 ms
Wall time: 2.2 s


In [81]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print("table: %s" % table_name)


schema: information_schema
table: sql_features
table: sql_implementation_info
table: sql_parts
table: sql_sizing
schema: public
table: prescriptor
table: client
table: test
table: medicament
table: beneficiary
table: medication_specialisation
table: medication_presentation
table: global_expenses
table: generic_group
